<a href="https://colab.research.google.com/github/wjdqlsdlsp/AI_using_pytorch-reinforce-learning/blob/main/%E1%84%80%E1%85%AA%E1%84%8C%E1%85%A65_Cliff_Walking_with_DQN_%E1%84%87%E1%85%A1%E1%86%A8%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%87%E1%85%B5%E1%86%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[인공지능] 과제5 Cliff Walking 예제 DQN 구현**
*   **DQN Class를 완성하여 결과를 살펴보는 것이 목표**입니다.
*   기본적인 코드는 아래 노트에 모두 작성되어 있습니다. 비어있는 함수 부분을 완성하면 됩니다.
*   **과제 수행 시 주의사항: 외부 라이브러리로 DQN 적용하지 말 것, 수업 때 배운 내용대로 DQN을 주어진 함수에 구현할 것.** 웹 상에 있는 다양한 DQN 코드를 참고하는 것은 괜찮습니다.
*   **보고서 작성 내용**: 여러분이 완성한 DQN 알고리즘의 내용과 결과의 의미를 분석하는 내용을 작성하면 됩니다.
작성한 코드와 실행 결과를 첨부하길 바라며, 코드에는 자세한 주석을 필수적으로 포함하기 바랍니다. 보고서는 PDF로 제출바랍니다.
*   보고서는 12월 16일 오후 11시 59분까지 블랙보드에 보고서 형태로 제출하면 됩니다. 지각은 0점입니다.
*   **Deep Sarsa를 추가로 구현하여 보고서에 관련 내용을 추가적으로 작성 및 제출할 경우 가산점 5점 부여**

# **본 노트를 본인의 drive로 복사하여 활용하기 바랍니다.**

본 과제 역시 이전과제와 동일한 환경을 사용합니다. 따라서 학습된 Q-value를 입력하면 해당하는 Q-value의 greedy 정책이 출력되도록 하는 QtoPolicy Class 또한 동일하게 사용됩니다.

In [ ]:
import numpy as np
import random
from tqdm import tqdm
from collections import defaultdict, namedtuple, deque
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv # Cliff Walking 환경

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class QtoPolicy:
    def __init__(self):
        self.action = ['↑', '→', '↓', '←', 'X']

    def printPolicy(self, Q):
        policy = np.array([np.argmax(Q[key]) if key in Q else -1 for key in np.arange(48)])
        v = ([np.max(Q[key]) if key in Q else 0 for key in np.arange(48)])
        actions = np.stack([self.action for _ in range(len(policy))], axis=0)
        policy[36:] = np.array([0] + [3] * 10 + [4])

        print(np.take(actions, np.reshape(policy, (4, 12))))
        print('')

DQN의 experience replay기법 구현을 위한 replay buffer class를 생성합니다. DQN에서는 (state, action, reward, next_state)가 저장됩니다.

In [ ]:
# Experience replay를 위한 replay buffer class 생성
Transition = namedtuple('Transition',
                        ('state', 'action', 'reward', 'next_state'))


class ReplayBuffer(object):

    def __init__(self, capacity):
        self.buffer = deque([],maxlen=capacity)

    def push(self, *args):
        self.buffer.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)

DQN에서 Q-Network으로 사용할 신경망 모델을 PyTorch 기반으로 정의합니다.

In [ ]:
class DNN(nn.Module):
    # DNN 모델 설계 및 초기값 설정
    def __init__(self, inputs, outputs):
        super(DNN, self).__init__()
        self.x_dim = inputs
        self.y_dim = outputs
        self.fc_variable_no = 100

        # network 용 변수
        self.fc_in = nn.Linear(self.x_dim, self.fc_variable_no)
        self.fc_hidden1 = nn.Linear(self.fc_variable_no, self.fc_variable_no)
        self.fc_hidden2 = nn.Linear(self.fc_variable_no, self.fc_variable_no)
        self.fc_hidden3 = nn.Linear(self.fc_variable_no, self.fc_variable_no)
        self.fc_out = nn.Linear(self.fc_variable_no, self.y_dim)
        self.relu = nn.ReLU()

    # 전파 과정
    def forward(self, x):
        x = torch.reshape(x, [-1, self.x_dim])
        x = self.relu(self.fc_in(x))
        x = self.relu(self.fc_hidden1(x))
        x = self.relu(self.fc_hidden2(x))
        x = self.relu(self.fc_hidden3(x))
        x = self.fc_out(x)
        return x

DQN 알고리즘 class를 정의합니다. 하이퍼파라미터는 주어진 값을 사용하면 됩니다.

In [ ]:
class DQN:
    def __init__(self):
        self.state_no = 48 # state의 갯수
        self.action_no = 4 # action의 갯수
        self.alpha = 0.001 # 학습률
        self.gamma = 0.99 # Discount factor
        self.epsilon = 0.2 # 앱실론

        
        self.batch_size = 32  # Experience replay에서의 batch size
        self.training_interval = 10  # Q-Network 학습 interval
        self.target_update_interval = 100  # target Q-Network 학습 interval

        self.main_net = DNN(self.state_no, self.action_no) # DNN 모델
        self.target_net = DNN(self.state_no, self.action_no) # DNN 모델 (타겟 계산)

        # Fixed target Q-Network를 정의하고 main Q-network와 동일하게 초기화
        self.target_net.load_state_dict(self.main_net.state_dict())
        self.target_net.eval()

        self.optimizer = optim.Adam(self.main_net.parameters(), lr=self.alpha)
        # Experience replay를 위한 buffer 정의
        self.buffer = ReplayBuffer(500)

    # state의 인덱스가 연속적인 의미를 가지고 있지 않으므로 효과적인 학습을
    # 위해 one-hot encoding을 수행
    def one_hot_state(self, state):
        one_hot_encoded = np.zeros((1, self.state_no))
        one_hot_encoded[0, state] = 1

        return one_hot_encoded
    
    # 학습이 끝난 후 Q-Network에서 Q-value 계산하는 함수
    def get_q_values(self):
        q_values = defaultdict(lambda: [0.0] * self.action_no)
        # 각 state 별 Q-value 계산
        for i in range(self.state_no):
            state = torch.tensor(self.one_hot_state(i)).float()
            q_values[i] = self.main_net(state).tolist()
        return q_values

    # 신경망 최적화 모델
    def optimize_model(self):
        # 버퍼의 크기가 배치사이즈보다 작을 경우, return
        if len(self.buffer) < self.batch_size:
            return
        
        # 학습을 위한 transition의 랜덤 배치를 선택
        transitions = self.buffer.sample(self.batch_size)
        
        # 위에서 정의한 Transition을 이용하여 각각의 이름을 설정
        batch = Transition(*zip(*transitions))

        # batch.next_state값을 map을 이용하여 다음으로 변환 : None 여부 판단
        non_final_mask = torch.tensor(tuple(map(lambda s : s is not None,
                                                batch.next_state)), dtype=torch.bool)
        # torch.cat을 이용하여 tensor를 연결
        non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)

        # 밸류 값 계산을 위해 main_net이용. Q(s)를 계산한 뒤, 이에 대한 action의 열을 선택
        state_action_values = self.main_net(state_batch).gather(1, action_batch)
        
        # target 값 계산을 위한 값 0으로 선언
        next_state_values = torch.zeros(self.batch_size)

        '''
        타겟 값 계산을 위해 target_net을 이용. 위에서 정의한 non_final_next_states를 입력
        이를 통해, 다음 state에 대한 Q(s')를 구함
        '''
        next_state_values[non_final_mask] = self.target_net(non_final_next_states).max(1)[0].detach()
        # 공식에 의해서, 타겟 값 계산
        target_state_action_values = (next_state_values * self.gamma) + reward_batch

        # 손실 함수 정의
        criterion = nn.SmoothL1Loss()
        loss = criterion(state_action_values, target_state_action_values)

        # 오류 역전파 진행
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.main_net.parameters():
            param.grad.data.clamp_(-10, 10)
        # 그레디언트 갱신
        self.optimizer.step()

    # DQN 갱신 부분
    def update(self, state, action, reward, next_state, time_step):
        # 버퍼에 값 저장
        self.buffer.push(torch.from_numpy(state).float(),
                         torch.tensor(action).reshape((-1, 1)),
                         torch.tensor(reward).reshape((-1, 1)),
                         torch.from_numpy(next_state).float())
        # train_interval 마다, DNN 최적화 함수 실행
        if (time_step + 1) % self.training_interval == 0:
            self.optimize_model()
        # target_update_interval 마다, 타겟넷의 값을 main_net 값으로 갱신 
        if (time_step + 1) % self.target_update_interval == 0:
            self.target_net.load_state_dict(self.main_net.state_dict())


    # DQN epsilon-greedy 정책
    def act(self, state):
        # 설정한 입실론 값보다 작을 경우, 랜덤 액션을 취함
        if np.random.rand() < self.epsilon:
            action = np.random.choice(self.action_no)
        # 설정한 입실론 값보다 클 경우
        else:
            with torch.no_grad():
                # 모델에 입력하기 위해 텐서 변환
                state = torch.from_numpy(state).float()
                # main_net을 통해 q_value값을 얻음
                q_values = self.main_net(state)
                # q_value값이 가장 큰 값인 행동을 선택하여 return
                action = torch.argmax(q_values, 1).item()
        return action


이전 과제와 동일하게 OpenAI Gym에서의 Cliff Walking 환경을 로드하고, 주어진 Q-value에서 greedy policy를 출력하는 QtoPolicy Class를 정의합니다.


In [ ]:
env = CliffWalkingEnv()
policy = QtoPolicy()

DQN Class를 정의하고 5000 episode 동안 학습을 수행합니다.

DQN에서의 training interval 및 target Q-network update interval을 위해 episode와 관계없는 time-step을 사용하여 DQN class에서 활용할 수 있게 해줍니다.

In [ ]:
agent_DQN = DQN()
time_step = 0
for ep in tqdm(range(5000)):
    done = False
    state = env.reset()
    state = agent_DQN.one_hot_state(state)
    action = agent_DQN.act(state)
    ep_reward = 0
    ep_steps = 0
    if ep % 50 ==0:
        print("\n")
        policy.printPolicy(agent_DQN.get_q_values())
    while not done:
        next_state, reward, done, info = env.step(action)

        next_state = agent_DQN.one_hot_state(next_state)

        next_action = agent_DQN.act(next_state)

        agent_DQN.update(state, action, reward, next_state, time_step)
        time_step = time_step + 1

        ep_reward+=reward
        state = next_state
        action = next_action
        ep_steps = ep_steps + 1

  0%|          | 0/5000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:912: UserWarning: Using a target size (torch.Size([32, 32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)




[['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→']
 ['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→']
 ['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→']
 ['↑' '←' '←' '←' '←' '←' '←' '←' '←' '←' '←' 'X']]



  1%|          | 52/5000 [00:40<12:34,  6.56it/s]



[['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→']
 ['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→']
 ['→' '→' '→' '→' '→' '→' '↓' '→' '↓' '→' '→' '↓']
 ['↑' '←' '←' '←' '←' '←' '←' '←' '←' '←' '←' 'X']]



  2%|▏         | 101/5000 [00:53<13:29,  6.05it/s]

...

 98%|█████████▊| 4901/5000 [15:20<00:15,  6.51it/s]



[['↓' '↓' '↓' '↓' '↓' '↓' '↓' '↓' '↓' '↓' '↓' '↓']
 ['↓' '↓' '↓' '↓' '↓' '↓' '↓' '↑' '↓' '↓' '↓' '↓']
 ['↓' '↑' '↑' '↑' '↑' '↑' '↑' '↑' '↑' '↑' '↑' '↓']
 ['↑' '←' '←' '←' '←' '←' '←' '←' '←' '←' '←' 'X']]



 99%|█████████▉| 4951/5000 [15:30<00:14,  3.34it/s]



[['→' '→' '→' '→' '↓' '→' '→' '↓' '→' '→' '→' '→']
 ['→' '→' '→' '→' '↓' '→' '→' '→' '↓' '→' '→' '↓']
 ['→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '↓']
 ['↑' '←' '←' '←' '←' '←' '←' '←' '←' '←' '←' 'X']]



100%|██████████| 5000/5000 [15:39<00:00,  5.32it/s]


학습된 Q-value를 이용하여 학습된 정책을 출력합니다.

In [ ]:
print('Learned policy by DQN')
policy.printPolicy(agent_DQN.get_q_values())

Learned policy by DQN
[['→' '↓' '→' '→' '↓' '→' '↓' '↓' '↓' '→' '→' '→']
 ['→' '→' '→' '→' '↓' '↓' '↓' '→' '↓' '↓' '→' '↓']
 ['↓' '→' '→' '→' '→' '→' '→' '→' '→' '→' '→' '↓']
 ['↑' '←' '←' '←' '←' '←' '←' '←' '←' '←' '←' 'X']]



In [ ]:
env.close()